<a href="https://colab.research.google.com/github/thiagoneye/master-sciml/blob/main/04-neural_solvers/notebooks/heat_equation_dirichlet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## PINN para a Equação do Calor 1D

Este notebook implementa uma **Rede Neural Informada pela Física (PINN)** para resolver a Equação do Calor 1D.

A abordagem combina o aprendizado supervisionado tradicional com a imposição de uma restrição física (a própria equação diferencial parcial), permitindo que a rede neural aprenda a solução do problema em todo o domínio espaço-tempo.

### A Equação do Calor 1D Transiente

A equação governante do fenômeno é:

$$ \frac{\partial T}{\partial t} = \alpha \frac{\partial^2 T}{\partial x^2} $$

Onde:
- $T$ é a temperatura.
- $t$ é o tempo.
- $x$ é a posição.
- $\alpha$ é a difusividade térmica do material (uma constante).

A rede neural será treinada para aproximar a função $T(x, t)$. A função de perda total será uma combinação de:

1.  **Perda de Dados (MSE):** O erro entre a previsão da rede e os dados de medição fornecidos.
2.  **Perda da Física (Residual da PDE):** Uma medida de quanto a saída da rede viola a equação do calor.

### Resumo do Procedimento

1.  **Carregar e Preparar Dados**: Carregue seu `df` e crie tensores para as posições `(x, t)` e temperaturas `T`.
2.  **Gerar Pontos de Colocação**: Crie um conjunto de pontos `(x, t)` aleatórios dentro do seu domínio para avaliar a perda da física.
3.  **Definir o Modelo**: Crie uma rede neural sequencial com Keras que mapeia `(x, t)` para `T`.
4.  **Implementar as Funções de Perda**:
    - `data_loss`: MSE entre as previsões e os dados reais.
    - `physics_loss`: MSE do residual da PDE, calculado usando `tf.GradientTape` para obter as derivadas.
5.  **Criar o Loop de Treinamento**:
    - Em cada época, calcule a perda de dados e a perda da física.
    - Some-as para obter a perda total.
    - Use `tf.GradientTape` para encontrar os gradientes da perda total em relação aos pesos do modelo.
    - Aplique esses gradientes usando um otimizador como o Adam.
6.  **Treinar e Avaliar**: Execute o loop por um número suficiente de épocas e, ao final, use o `pinn_model` treinado para fazer previsões e visualizar a solução.

### Importações

In [ ]:
import time

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
import tensorflow_probability as tfp

from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

tf.keras.backend.set_floatx("float64")
tf.random.set_seed(42)
np.random.seed(42)

print(f"TensorFlow Version: {tf.__version__}")

gpus = tf.config.list_physical_devices("GPU")
print(f"Número de GPUs disponíveis: {len(gpus)}")
if gpus:
    print("A GPU está sendo utilizada pelo TensorFlow.")
    print("Detalhes da GPU:", gpus)
else:
    print("O TensorFlow NÃO encontrou a GPU.")

### Leitura e Preparação dos Dados

In [ ]:
# Parâmetros do problema
ALPHA = 0.5

# Leitura dos dados
df_full = pd.read_csv("heat_equation_dirichlet.csv")

# Dados completos, entrada (x, t) e saída (T)
X = df_full[["time", "position"]].values
T = df_full[["temperature"]].values

# Dados de treino
df = df_full[
    (df_full["time"] == 0) | (df_full["position"] == 0) | (df_full["position"] == 1)
]
df = df.sample(frac=0.665, random_state=42)

X_train_raw = df[["time", "position"]].values
T_train_raw = df[["temperature"]].values

# Converte para tensores do TensorFlow
X_train_tf = tf.convert_to_tensor(X_train_raw, dtype=tf.float64)
T_train_tf = tf.convert_to_tensor(T_train_raw, dtype=tf.float64)

# Plot
fig, ax = plt.subplots(figsize=(6, 3))
ax.scatter(X_train_raw[:, 0], X_train_raw[:, 1], alpha=0.3, label=f"{len(df)} pontos")
ax.set_title("Pontos de Treinamento", fontsize=14)
ax.set_xlabel("Tempo (t)", fontsize=10)
ax.set_ylabel("Posição (x)", fontsize=10)
ax.legend()
plt.savefig("../results/dirichlet_condition/training_points.png", dpi=600)

### Geração dos Pontos de Colocação

Estes são os pontos no interior do domínio onde não conhecemos a temperatura. Eles serão usados para calcular a **perda da física** ($L_{física}$), forçando a rede a obedecer à equação do calor.

In [ ]:
N_collocation = 20000

# Pega os limites do domínio a partir dos dados
t_min, t_max = df_full["time"].min(), df_full["time"].max()
x_min, x_max = df_full["position"].min(), df_full["position"].max()

# Gera pontos aleatórios dentro desses limites
x_collocation = tf.random.uniform(
    (N_collocation, 1), minval=x_min, maxval=x_max, dtype=tf.float64
)
t_collocation = tf.random.uniform(
    (N_collocation, 1), minval=t_min, maxval=t_max, dtype=tf.float64
)

X_collocation_raw = tf.concat([t_collocation, x_collocation], axis=1)
X_collocation_tf = tf.convert_to_tensor(X_collocation_raw, dtype=tf.float64)

# Plot
fig, ax = plt.subplots(figsize=(6, 3))
ax.scatter(
    X_collocation_raw[:, 0],
    X_collocation_raw[:, 1],
    alpha=0.3,
    label=f"{N_collocation} pontos",
)
ax.set_title("Pontos de Colocação", fontsize=14)
ax.set_xlabel("Tempo (t)", fontsize=10)
ax.set_ylabel("Posição (x)", fontsize=10)
ax.legend()
plt.savefig("../results/dirichlet_condition/placement_points.png", dpi=600)

### Construção do Modelo (Rede Neural)

In [ ]:
def create_pinn_model(num_hidden_layers=10, num_neurons=64):
    model = tf.keras.Sequential()

    model.add(tf.keras.layers.InputLayer(input_shape=(2,)))

    for _ in range(num_hidden_layers):
        model.add(tf.keras.layers.Dense(num_neurons, activation="tanh"))

    model.add(tf.keras.layers.Dense(1, activation=None))

    return model


pinn_model = create_pinn_model()
pinn_model.summary()

### Função de Perda Híbrida

A função de perda total é a soma da perda nos dados e da perda na física. Aqui, implementamos o cálculo do resíduo da EDP.

In [ ]:
def data_loss(model, X_data, T_data):
    T_pred = model(X_data)
    return tf.reduce_mean(tf.square(T_pred - T_data))


def physics_loss(model, t, x, ALPHA):
    with tf.GradientTape(persistent=True) as tape:
        tape.watch(t)
        tape.watch(x)

        X_concat = tf.concat([t, x], axis=1)
        T_pred = model(X_concat)

        dT_dx = tape.gradient(T_pred, x)

    dT_dt = tape.gradient(T_pred, t)
    d2T_dx2 = tape.gradient(dT_dx, x)

    del tape

    residual = dT_dt - ALPHA * d2T_dx2
    return tf.reduce_mean(tf.square(residual))

### Loop de Treinamento Customizado

Usamos um loop customizado para calcular as perdas e aplicar os gradientes a cada época.

In [ ]:
W_DATA = 1.0
W_PHYSICS = 1.0

optimizer = tf.keras.optimizers.Adam(learning_rate=1e-4)
EPOCHS_ADAM = 200000


@tf.function
def train_step(model, X_data, T_data, X_collocation):
    with tf.GradientTape() as tape:
        loss_d = data_loss(model, X_data, T_data)
        loss_p = physics_loss(
            model, X_collocation[:, 0:1], X_collocation[:, 1:2], ALPHA
        )
        total_loss = W_DATA * loss_d + W_PHYSICS * loss_p

    gradients = tape.gradient(total_loss, model.trainable_variables)
    optimizer.apply_gradients(zip(gradients, model.trainable_variables))
    return loss_d, loss_p, total_loss


history = []
start_time = time.time()
print("Iniciando Fase 1: Treinamento com Adam...")

for epoch in range(EPOCHS_ADAM):
    loss_d, loss_p, total_loss = train_step(
        pinn_model, X_train_tf, T_train_tf, X_collocation_tf
    )

    current_time = time.time() - start_time
    history.append([epoch, loss_d, loss_p, total_loss])

    if (epoch + 1) % 500 == 0:
        print(
            f"Epoch {epoch+1:5d}, Loss_Data: {loss_d:.4e}, Loss_Physics: {loss_p:.4e}, Loss_Total: {total_loss:.4e}"
        )

print(f"Fase 1 (Adam) concluída em {current_time:.2f} segundos.")

### Log de Treinamento

In [ ]:
history = np.array(history)

plt.figure(figsize=(10, 6))
plt.plot(history[:, 0], history[:, 1], label="Perda dos Dados")
plt.plot(history[:, 0], history[:, 2], label="Perda da Física")
plt.plot(history[:, 0], history[:, 3], label="Perda Total")
plt.xlabel("Épocas")
plt.ylabel("Perda")
plt.yscale("log")
plt.title("Log de Treinamento")
plt.legend()
plt.savefig("../results/dirichlet_condition/training_log.png", dpi=600)

In [ ]:
# Função auxiliar para definir os pesos a partir de um vetor 1D
def set_weights(model, weights_1d):
    """
    Desachata um vetor 1D de pesos e os atribui às variáveis do modelo.
    Esta função corrige a lógica de unflattening.
    """
    shapes = [v.shape for v in model.trainable_variables]
    sizes = [tf.reduce_prod(s) for s in shapes]

    start = 0
    for i, (shape, size) in enumerate(zip(shapes, sizes)):
        end = start + size
        # Pega a fatia do vetor 1D e remodela para a forma original da camada
        w = tf.reshape(weights_1d[start:end], shape)
        model.trainable_variables[i].assign(w)
        start = end


def loss_and_grads_func(weights_1d):
    """
    Calcula a perda e os gradientes para um dado vetor 1D de pesos.
    """
    # Atribui os pesos do vetor 1D ao modelo.
    set_weights(pinn_model, tf.cast(weights_1d, dtype=tf.float64))

    # Calcula a perda com os novos pesos
    with tf.GradientTape() as tape:
        loss_d = data_loss(pinn_model, X_train_tf, T_train_tf)
        loss_p = physics_loss(
            pinn_model, X_collocation_tf[:, 0:1], X_collocation_tf[:, 1:2], ALPHA
        )
        total_loss = W_DATA * loss_d + W_PHYSICS * loss_p

    # Calcula os gradientes
    grads = tape.gradient(total_loss, pinn_model.trainable_variables)

    # Achata a lista de gradientes em um único vetor 1D
    grads_1d = tf.concat([tf.reshape(g, [-1]) for g in grads], axis=0)

    return tf.cast(total_loss, tf.float64), tf.cast(grads_1d, tf.float64)


# Execução da otimização L-BFGS

# Pega os pesos atuais do modelo (após Adam) e os achata em um vetor 1D
initial_weights = tf.concat(
    [tf.reshape(v, [-1]) for v in pinn_model.trainable_variables], axis=0
)

start_time = time.time()
print("\nIniciando Fase 2: Refinamento com L-BFGS...")

# Chama o otimizador L-BFGS
results = tfp.optimizer.lbfgs_minimize(
    value_and_gradients_function=loss_and_grads_func,
    initial_position=tf.cast(initial_weights, tf.float64),
    num_correction_pairs=50,
    max_iterations=2000,
    tolerance=1e-10,
)
lbfgs_time = time.time() - start_time
print(f"Fase 2 (L-BFGS) concluída em {lbfgs_time:.2f} segundos.")

# Atribuição final dos pesos otimizados
optimized_weights_1d = results.position
set_weights(pinn_model, tf.cast(optimized_weights_1d, dtype=tf.float64))

# Imprime a perda final após L-BFGS
final_loss_d = data_loss(pinn_model, X_train_tf, T_train_tf)
final_loss_p = physics_loss(
    pinn_model, X_collocation_tf[:, 0:1], X_collocation_tf[:, 1:2], ALPHA
)
final_total_loss = W_DATA * final_loss_d + W_PHYSICS * final_loss_p

print("\nTreinamento concluído.")
print(
    f"Perda Final (após L-BFGS): Loss_Data: {final_loss_d:.4e}, Loss_Physics: {final_loss_p:.4e}, Loss_Total: {final_total_loss:.4e}"
)

### Métricas de Avaliação

Agora, usamos o modelo treinado para prever a temperatura em todo o domínio e comparamos com os dados de treinamento originais.

In [ ]:
# Fazer previsões nos mesmos pontos do DataFrame de treinamento
T_pred = pinn_model.predict(X)

# Calcular as métricas de erro
mse = mean_squared_error(T, T_pred)
mae = mean_absolute_error(T, T_pred)
r2 = r2_score(T, T_pred)

print(f"Erro Quadrático Médio (MSE): {mse:.6f}")
print(f"Erro Absoluto Médio (MAE):  {mae:.6f}")
print(f"R2 Score:  {r2:.6f}")

### Visualização dos Resultados

In [ ]:
t = np.unique(X[:, 0])
x = np.unique(X[:, 1])

Nt = len(t)
Nx = len(x)

U_sol = T.reshape([Nt, Nx])
U_pred = T_pred.reshape([Nt, Nx])
Err = np.abs(U_sol - U_pred)

T_grid, X_grid = np.meshgrid(t, x)

# Mapa de Calor U(x, t) - Valores Reais

plt.figure(figsize=(6, 3))
ax = plt.gca()

im = plt.pcolormesh(T_grid, X_grid, U_sol.T, cmap="coolwarm", shading="gouraud")
cbar = fig.colorbar(im, ax=ax)
cbar.set_label("Temperatura (U)", fontsize=10)

ax.set_title("Solução U(x, t)")
ax.set_xlabel("Tempo (t)", fontsize=10)
ax.set_ylabel("Posição (x)", fontsize=10)

plt.savefig("../results/dirichlet_condition/true_solution.png", dpi=600)

# Mapa de Calor U(x, t) - Valores Previstos

plt.figure(figsize=(6, 3))
ax = plt.gca()

im = plt.pcolormesh(T_grid, X_grid, U_pred.T, cmap="coolwarm", shading="gouraud")
cbar = fig.colorbar(im, ax=ax)
cbar.set_label("Temperatura (U)", fontsize=10)

ax.set_title("Predição U(x, t)")
ax.set_xlabel("Tempo (t)", fontsize=10)
ax.set_ylabel("Posição (x)", fontsize=10)

plt.savefig("../results/dirichlet_condition/prediction_solution.png", dpi=600)

# Mapa de Calor do Erro Err(x, t)

plt.figure(figsize=(6, 3))
ax = plt.gca()

im = plt.pcolormesh(T_grid, X_grid, Err.T, cmap="coolwarm", shading="gouraud")
cbar = fig.colorbar(im, ax=ax)
cbar.set_label("Erro Absoluto", fontsize=10)

ax.set_title("Erro de Predição")
ax.set_xlabel("Tempo (t)", fontsize=10)
ax.set_ylabel("Posição (x)", fontsize=10)

plt.savefig("../results/dirichlet_condition/error.png", dpi=600)